In [1]:
!pip install -q -U watermark


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
pip install -q efficient_apriori

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import efficient_apriori
import matplotlib.pyplot as plt
from datetime import datetime
from itertools import combinations
from efficient_apriori import apriori
import warnings
warnings.filterwarnings('ignore')

In [4]:
%reload_ext watermark
%watermark -a "Juliana Santimaria" 

Author: Juliana Santimaria



In [5]:
dados_departamentos = pd.read_csv('departamentos.csv')
dados_corredores = pd.read_csv('corredores.csv')
dados_produtos = pd.read_csv('produtos.csv')
dados_pedidos = pd.read_csv('pedidos.csv')
dados_transacoes = pd.read_csv('transacoes.csv')

In [6]:
conta_arrays = np.array([['dados_departamentos', len(dados_departamentos)],
                        ['dados_corredores', len(dados_corredores)],
                        ['dados_produtos', len(dados_produtos)],
                        ['dados_pedidos', len(dados_pedidos)],
                        ['dados_transacoes', len(dados_transacoes)]])

In [7]:
contagem = pd.DataFrame(conta_arrays, columns = ['Arquivo', 'Total de Registros'])

In [8]:
print(contagem)

               Arquivo Total de Registros
0  dados_departamentos                 21
1     dados_corredores                134
2       dados_produtos              49688
3        dados_pedidos            3421083
4     dados_transacoes           32434489


In [9]:
dados_departamentos.shape

(21, 2)

In [10]:
dados_departamentos.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [11]:
dados_corredores.shape

(134, 2)

In [12]:
dados_corredores.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [13]:
dados_produtos.shape

(49688, 4)

In [14]:
dados_produtos.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [15]:
dados_pedidos.shape

(3421083, 7)

In [16]:
dados_pedidos.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [17]:
dados_transacoes.shape

(32434489, 4)

In [18]:
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [19]:
dados_departamentos.isna().sum()

department_id    0
department       0
dtype: int64

In [20]:
dados_corredores.isna().sum()

aisle_id    0
aisle       0
dtype: int64

In [21]:
dados_produtos.isna().sum()

product_id       0
product_name     0
aisle_id         0
department_id    0
dtype: int64

In [22]:
dados_pedidos.isna().sum()

order_id                       0
user_id                        0
eval_set                       0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [23]:
dados_transacoes.isna().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

In [24]:
%%time
dados_transacoes = dados_transacoes.merge(dados_pedidos, on = 'order_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_produtos, on = 'product_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_corredores, on = 'aisle_id', how = 'left')
dados_transacoes = dados_transacoes.merge(dados_departamentos, on = 'department_id', how = 'left')

CPU times: user 12.5 s, sys: 12.6 s, total: 25.1 s
Wall time: 29.1 s


In [25]:
dados_transacoes.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


In [26]:
dados_transacoes.isna().sum()

order_id                        0
product_id                      0
add_to_cart_order               0
reordered                       0
user_id                         0
eval_set                        0
order_number                    0
order_dow                       0
order_hour_of_day               0
days_since_prior_order    2078068
product_name                    0
aisle_id                        0
department_id                   0
aisle                           0
department                      0
dtype: int64

In [27]:
dados_transacoes.isnull().sum() / len(dados_transacoes) * 100

order_id                  0.00000
product_id                0.00000
add_to_cart_order         0.00000
reordered                 0.00000
user_id                   0.00000
eval_set                  0.00000
order_number              0.00000
order_dow                 0.00000
order_hour_of_day         0.00000
days_since_prior_order    6.40697
product_name              0.00000
aisle_id                  0.00000
department_id             0.00000
aisle                     0.00000
department                0.00000
dtype: float64

In [28]:
%%time
df_mod = pd.DataFrame(dados_transacoes.groupby('order_id')['product_id'])

CPU times: user 1min 4s, sys: 1min 2s, total: 2min 6s
Wall time: 2min 29s


In [29]:
df_mod.shape

(3214874, 2)

In [30]:
df_mod.head()

,0,1
0,2,0 33120 1 28985 2 9327 3 45918 4 ...
1,3,9 33754 10 24838 11 17704 12 2190...
2,4,17 46842 18 26434 19 39758 20 2776...
3,5,30 13176 31 15005 32 47329 33 2796...
4,6,56 40462 57 15873 58 41897 Name: prod...


In [31]:
prod = []

In [32]:
%%time
for i in range(len(df_mod[0])):
    prod.append(list(df_mod.iloc[i][1]))

CPU times: user 2min 7s, sys: 5.46 s, total: 2min 12s
Wall time: 2min 15s


In [33]:
transacoes = pd.DataFrame({'Id_Pedido' : df_mod[0], 'Produtos' : prod})

In [34]:
transacoes.head(10)

,Id_Pedido,Produtos
0,2,"[33120, 28985, 9327, 45918, 30035, 17794, 4014..."
1,3,"[33754, 24838, 17704, 21903, 17668, 46667, 174..."
2,4,"[46842, 26434, 39758, 27761, 10054, 21351, 225..."
3,5,"[13176, 15005, 47329, 27966, 23909, 48370, 132..."
4,6,"[40462, 15873, 41897]"
5,7,"[34050, 46802]"
6,8,[23423]
7,9,"[21405, 47890, 11182, 2014, 29193, 34203, 1499..."
8,10,"[24852, 4796, 31717, 47766, 4605, 1529, 21137,..."
9,11,"[30162, 27085, 5994, 1313, 31506]"


In [35]:
%watermark -a "Juliana Santimaria"

Author: Juliana Santimaria



In [36]:
#%watermark -v -m

In [37]:
#%watermark --iversions